# Matching two list of author's name for publication
One list of author's name with email needs to be updated with current list of authors on our recent publication. The list needs to be cleaned and linked.

In [123]:
import pandas as pd
import re

censor = True

#### Get current list of authors in a pandas dataframe

In [124]:
char = '''Joshua T. Vogelstein1YB; Timothy Verstynen3YB; Konrad P. Kording2YB;
Leyla Isik1Y; John W. Krakauer1; Ralph Etienne-Cummings1; Elizabeth L. Ogburn1; Carey E. Priebe1;
Randal Burns1; Kwame Kutten1; James J. Knierim1; James B. Potash1; Thomas Hartung1; Lena 
Smirnova1; Alena Savonenko1; Ian Phillips1; Michael I. Miller1; Rene Vidal1; Jeremias Sulam1; Adam 
Charles1; Noah J. Cowan1; Maxim Bichuch1; Archana Venkataraman1; Chen Li1; Nitish Thakor1;
Justus M Kebschull1; Marilyn Albert1; Jinchong Xu1; Marshall Hussain Shuler1; Brian Caffo1; Tilak 
Ratnanather1; Ali Geisa1; Seung-Eon Roh1; Eva Yezerets1; Meghana Madhyastha1; Javier J. How1;
Tyler M. Tomita1; Jayanta Dey1; Ningyuan (Teresa) Huang1; Ashwin De Silva1; Jong M. Shin1; Kaleab 
Alemayehu Kinfu1; Soledad Villar1; Pratik Chaudhari2; Ben Baker2; Anna C. Schapiro2; Dinesh Jayaraman2;
Eric Eaton2; Michael Platt2; Lyle Ungar2; Leila Wehbe3; Michael J. Tarr3; Adam Kepecs4; Amy Christensen4;
Onyema Osuagwu5; Bing Brunton6; Brett Mensh7; Alysson R. Muotri8; Gabriel Silva8; Francesca 
Puppo8; Florian Engert9; Elizabeth Hillman10; Julia Brown11; Chris White12; Weiwei Yang12; Andrei 
A. Rusu13'''

In [125]:
tmp = char.replace('\n', '').split(';')
tmp[0].rfind('[0-9]')
tmp[0]

'Joshua T. Vogelstein1YB'

In [126]:
df = pd.DataFrame(tmp, columns=['Name'])

In [127]:
def remove_int_beyond(xx):
    out = []
    for x in xx:
        s = re.search('\d', x).start()
        out.append(x[:s].strip())
    return out

In [128]:
curr_df = df.apply(lambda x: remove_int_beyond(x),axis=0)
curr_df

,Name
0,Joshua T. Vogelstein
1,Timothy Verstynen
2,Konrad P. Kording
3,Leyla Isik
4,John W. Krakauer
...,...
61,Elizabeth Hillman
62,Julia Brown
63,Chris White
64,Weiwei Yang


#### Load previous author list which is now outdated

In [129]:
df = pd.read_excel('authors/email-response-list.xlsx')

#### Redact email for privacy

In [130]:
def redact(x):
    if isinstance(x, str):
        s = x.find('<')
        e = x.find('@')
        return x.replace(x[s+1:e], 'XXX')
    else:
        return ''

In [131]:
if censor:
    df.iloc[:,0] = df.apply(lambda x: redact(x.iloc[0]), axis=1)

#### preprocess before probablistic linking

In [132]:
newdf = df.iloc[:,:2].dropna()
newdf.head()

,JOVO EMAIL LIST,WHITE PAPER AUTHOR LIST
0,"Adam Charles <XXX@jhu.edu>,",Adam Charles
1,"Adam Kepecs <XXX@gmail.com>,",Adam Kepecs
2,"Alena Savonenko <XXX@jhmi.edu>,",Alena Savonenko
3,"Ali Geisa <XXX@gmail.com>,",Ali Geisa
4,"Amy Christensen <XXX@gmail.com>,",Amy Christensen


In [133]:
newlist = df.iloc[:,0].tolist()
newlist2 = []

for i in newlist:
    if isinstance(i, str):
        tmp = i.split(' <')
        if len(tmp) == 1:
            continue

        tmp[1] = tmp[1].replace('>,','')
        newlist2.append(tmp)

In [134]:
df_out = pd.DataFrame(newlist2, columns=['Name', 'Email'])
df_out.shape, df_out.head()

((56, 2),
               Name          Email
 0     Adam Charles    XXX@jhu.edu
 1      Adam Kepecs  XXX@gmail.com
 2  Alena Savonenko   XXX@jhmi.edu
 3        Ali Geisa  XXX@gmail.com
 4  Amy Christensen  XXX@gmail.com)

In [135]:
# df_out.to_excel('current_author_list.xlsx', index=False)

In [136]:
if not censor:
    df_out = pd.read_excel('authors/current_author_list.xlsx')

#### Join old author list with new author list
Old author list has a column for email whereas the new author list only has the name of authors

In [137]:
import fuzzymatcher

In [138]:
curr_df.head(5), df_out.head(5)

(                   Name
 0  Joshua T. Vogelstein
 1     Timothy Verstynen
 2     Konrad P. Kording
 3            Leyla Isik
 4      John W. Krakauer,
               Name          Email
 0     Adam Charles    XXX@jhu.edu
 1      Adam Kepecs  XXX@gmail.com
 2  Alena Savonenko   XXX@jhmi.edu
 3        Ali Geisa  XXX@gmail.com
 4  Amy Christensen  XXX@gmail.com)

In [139]:
match_result = fuzzymatcher.fuzzy_left_join(
    df_left=curr_df, df_right=df_out, 
    left_on='Name', right_on='Name', 
    left_id_col='Name', right_id_col='Name')

match_result.shape

(66, 6)

In [140]:
match_result.head()

,best_match_score,__id_left,__id_right,Name_left,Name_right,Email
0,NaN,Joshua T. Vogelstein,None,Joshua T. Vogelstein,NaN,NaN
1,0.341898,Timothy Verstynen,Timothy Verstynen,Timothy Verstynen,Timothy Verstynen,XXX@gmail.com
2,0.138672,Konrad P. Kording,Konrad Koerding,Konrad P. Kording,Konrad Koerding,XXX@gmail.com
3,0.270464,Leyla Isik,Leyla Isik,Leyla Isik,Leyla Isik,XXX@jhu.edu
4,0.263017,John W. Krakauer,John Krakauer,John W. Krakauer,John Krakauer,XXX@gmail.com


In [141]:
df_out = match_result[['Name_left', 'Email']]
df_out.columns=['Name','Email']
df_out.head()

,Name,Email
0,Joshua T. Vogelstein,NaN
1,Timothy Verstynen,XXX@gmail.com
2,Konrad P. Kording,XXX@gmail.com
3,Leyla Isik,XXX@jhu.edu
4,John W. Krakauer,XXX@gmail.com


In [142]:
if not censor:
    df_out.to_excel('authors/final_output_email_list.xlsx', index=False)

#### Manual entry of missing emails

In [143]:
df_linear = pd.read_excel('authors/final_output_email_list_modified.xlsx')

#### Linearize email for convenience

In [144]:
linear_txt = '; '.join(df_linear['Email'])

In [145]:
# output hidden for privacy
if not censor:
    linear_txt

In [146]:
# with open('for_email.txt', 'w', encoding='utf-8') as f:
#     f.write(linear_txt)

#### End of Notebook